In [71]:
%load_ext autoreload
%autoreload 2
import siren
from siren.SIREN_Controller import SIREN_Controller
import os
import pandas as pd
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
from SIREN_Simulation import PrepareSIRENInputFromLHCb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
siren_intput_file = "Data/SIREN_Input/LHC13_EPOSLHC_light_0.txt"
if not os.path.isfile(siren_intput_file):
    flux_data = PrepareSIRENInputFromLHCb("LHC13","EPOSLHC","light")
    flux_data.to_csv("Data/SIREN_Input/LHC13_EPOSLHC_light_0.txt",index=False)

In [175]:
# Number of events to inject
events_to_inject = int(1e6)

# Expeirment to run
experiment = "LakeGeneva"

# Define the controller
controller = SIREN_Controller(events_to_inject, experiment)

# Particle to inject
primary_type = siren.dataclasses.Particle.ParticleType.NuMu

cross_section_model = "CSMSDISSplines"

xsfiledir = siren.utilities.get_cross_section_model_path(cross_section_model)

# Cross Section Model
target_type = siren.dataclasses.Particle.ParticleType.Nucleon

DIS_xs = siren.interactions.DISFromSpline(
    os.path.join(xsfiledir, "dsdxdy_nu_CC_iso.fits"),
    os.path.join(xsfiledir, "sigma_nu_CC_iso.fits"),
    [primary_type],
    [target_type], "m"
)

primary_xs = siren.interactions.InteractionCollection(primary_type, [DIS_xs])
controller.SetInteractions(primary_xs)


In [176]:
panel_1 = controller.detector_model.GetSector("panel5_1")
panel_2 = controller.detector_model.GetSector("panel5_2")
panel_3 = controller.detector_model.GetSector("panel5_3")

def GetPanelIntersections(location, direction):
    _loc = controller.detector_model.GeoPositionToDetPosition(siren.detector.GeometryPosition(siren.math.Vector3D(location))).get()
    _dir = controller.detector_model.GeoDirectionToDetDirection(siren.detector.GeometryDirection(siren.math.Vector3D(direction))).get()
    return (panel_1.geo.ComputeIntersections(_loc, _dir),
            panel_2.geo.ComputeIntersections(_loc, _dir),
            panel_3.geo.ComputeIntersections(_loc, _dir))


In [177]:
# Primary distributions
primary_injection_distributions = {}
primary_physical_distributions = {}

primary_external_dist = siren.distributions.PrimaryExternalDistribution("Data/SIREN_Input/LHC13_EPOSLHC_light_0.txt")
primary_injection_distributions["external"] = primary_external_dist


fid_vol = controller.GetFiducialVolume()
position_distribution = siren.distributions.PrimaryBoundedVertexDistribution(fid_vol)
primary_injection_distributions["position"] = position_distribution

# SetProcesses
controller.SetProcesses(
    primary_type, primary_injection_distributions, primary_physical_distributions
)

controller.Initialize()

In [ ]:
events = controller.GenerateEvents()

In [ ]:
controller.SaveEvents("Data/SIREN_Input/LHC13_EPOSLHC_light_0",
                      save_int_probs=True,
                      save_int_params=True)

In [ ]:
data = ak.from_parquet("Data/SIREN_Input/LHC13_EPOSLHC_light_0.parquet")

In [ ]:
muon_momenta = np.array(np.squeeze(data.secondary_momenta[:,:,0]))
hadron_momenta = np.squeeze(data.secondary_momenta[:,:,1])
nu_momenta = np.squeeze(data.primary_momentum)
mu_vertex = np.squeeze(data.vertex)
muon_momentum = np.array(np.linalg.norm(muon_momenta[:,1:],axis=1))
mu_dir = muon_momenta[:,1:] / np.expand_dims(muon_momentum,-1)

In [ ]:
lumi = 3000 #HL

e_bins = np.logspace(1,4,30)
weights = np.squeeze(data.wgt) * lumi * 1000 * np.squeeze(data.int_probs)
plt.hist(nu_momenta[:,0],bins=e_bins,alpha=0.5,weights=weights)
plt.hist(muon_momenta[:,0],bins=e_bins,alpha=0.5,weights=weights)
plt.hist(hadron_momenta[:,0],bins=e_bins,alpha=0.5,weights=weights)
plt.loglog()

In [ ]:
panel_intersections = []
mu_x = []
mu_y = []
for mv,md in zip(mu_vertex,mu_dir):
    dz = panel_2.geo.placement.Position.GetZ() - mv[2]
    delx = dz/md[2]*md[0]
    dely = dz/md[2]*md[1]
    mu_x.append(mv[0] + delx)
    mu_y.append(mv[1] + dely)
    panel_intersections.append(GetPanelIntersections(mv,md))

In [ ]:


hit_map = np.sum([[len(arr)>0 for arr in ints]for ints in panel_intersections],axis=-1)
zbins = np.linspace(9200,19800,50)
xybins = np.linspace(-10,10,50)
plt.hist2d(np.array(mu_vertex[:,0]),
           np.array(mu_vertex[:,1]),
           bins = xybins)
plt.show()

plt.hist2d(np.array(mu_vertex[:,2]),
           np.array(mu_vertex[:,1]),
           bins = (zbins,xybins))
plt.show()

#plt.hist(np.array(mu_vertex[:,2]),bins=zbins)
weights = np.array(np.squeeze(data.wgt) * lumi * 1000 * np.squeeze(data.int_probs))
weights *= events_to_inject / 2e5 # correct for sampling

plt.hist(np.array(mu_vertex[:,2])[hit_map==0],bins=zbins,weights=weights[hit_map==0],histtype="step")
plt.hist(np.array(mu_vertex[:,2])[hit_map==3],bins=zbins,weights=weights[hit_map==3],histtype="step")
plt.semilogy()
plt.show()

plt.hist2d(mu_x,
           mu_y,
           bins = (xybins,xybins),
           weights=weights*(hit_map>0))
plt.show()

mu_radii = np.sqrt(np.array(mu_x)**2 + np.array(mu_y)**2)
plt.hist(mu_radii,bins=np.linspace(0,100,100))
plt.show()

weights = np.array(np.squeeze(data.wgt) * lumi * 1000 * np.squeeze(data.int_probs) * (hit_map>0))
weights *= events_to_inject / 2e5 # correct for sampling
plt.hist(nu_momenta[:,0],bins=e_bins,alpha=0.5,weights=weights)
plt.hist(muon_momenta[:,0],bins=e_bins,alpha=0.5,weights=weights)
plt.hist(hadron_momenta[:,0],bins=e_bins,alpha=0.5,weights=weights)
plt.loglog()